In [6]:
from selenium import webdriver
import datetime
import time
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from scrapy.selector import Selector
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
import os
import re
import shutil
import xlrd
import pymysql
from scrapy.selector import Selector
import re



def get_element(browser, by=By.XPATH, value="//body", times=3, timeout=8, poll_frequency=0.3, multi=False, check=False):
        """获取元素 v2.9.20170223
        @param browser: 浏览器对象
        @param by: 搜索元素方式
        @param value: 搜索元素方式的值
        @param times: 重试次数
        @param timeout: 超时秒数
        @param poll_frequency: 调用频率
        @param multi: 是否是获取多个元素，True-是，False-否
        @param check: 是否需要检查WebElement元素的有效性(位置不为0，尺寸不为0)，True-需要，False-不需要
        @return: 获取到的元素"""
        
        # 重试次数
        try_times = 1
        # 元素
        if multi:
            element = []
        else:
            element = None
        # 定位器
        locator = (by, value)
        
        wait_seconds = 0  # 等待秒数
        start_time = datetime.datetime.now()  # 开始时间
        end_time = datetime.datetime.now()  # 结束时间
        
        # 循环等待加载元素
        while not element and try_times <= times and wait_seconds <= timeout:
            # 元素未找到 并且 重试次数小于规定次数 并且 在超时范围内
            end_time = datetime.datetime.now()
            interval = end_time - start_time
            wait_seconds = interval.total_seconds()
            print("[BrowserHelper :: get_element]", "加载WebElement元素 ...", "[try_times] =", try_times, "[wait_seconds] =", wait_seconds, "[locator] =", locator)
            try:
                # 元素加载等待
                call = multi and EC.presence_of_all_elements_located or EC.presence_of_element_located
                webdirver_wait = WebDriverWait(driver=browser, timeout=timeout, poll_frequency=poll_frequency, ignored_exceptions=None)
                element = webdirver_wait.until(call(locator))
                
                # 检查WebElement元素的有效性(位置不为0，尺寸不为0)
                if check:
                    if element and element.location and element.location.get('x', 0) > 0 and element.location.get('y', 0) and \
                        element.size and element.size.get('width', 0) > 0 and element.size.get('height', 0) > 0 :
                        # WebElement元素有效
                        pass
                    else:
                        element = multi and [] or None
            
            except NoSuchElementException as e:
                print("[BrowserHelper :: get_element] No such element: %s = %s" % locator, e)
            except TimeoutException as e:
                print("[BrowserHelper :: get_element] load element: %s = %s timeout" % locator, e)
            
            print("[BrowserHelper :: get_element] waite for load element ...", "[try_times] =", try_times, "[element] =", (element and True or False))
            
            try_times += 1
        
        print("[BrowserHelper :: get_element] get element(%s = %s) %s!" % (by, value, element and "successfull" or "failed"))
        
        return element


def open_excel(excel_file):
    try:
        book = xlrd.open_workbook(excel_file)  #文件名，把文件与py文件放在同一目录下
    except:
        print("open excel file failed!")
    try:
        sheet = book.sheet_by_name(r"【生意参谋平台】1")   #execl里面的worksheet1
        return sheet
    except:
        print("locate worksheet in excel failed!")


def connect():
    #连接数据库
    try:
        db = pymysql.connect(host="192.168.0.23",user="root",
            passwd="njds@2019",
            db="bi",
            charset='utf8')
        
#         db = pymysql.connect(host="localhost",user="root",
#             passwd="aixocm123",
#             db="bi",
#             charset='utf8')
        
        return db
    except:
        print("could not connect to mysql server")


def init_chrome_browser():
    # 启动允许多个文件下载的浏览器
    options = webdriver.ChromeOptions()
    prefs = {"credentials_enable_service": False, "profile.password_manager_enabled": False, "profile.managed_default_content_settings.images": 1,'profile.default_content_settings.popups': 0, "profile.default_content_setting_values.automatic_downloads":1}
    options.add_experimental_option('prefs', prefs)
#     options.add_extension("C:\\Users\\xieruiping\\Desktop\\extension_0_0_1_1.crx");
#     options.add_experimental_option('excludeSwitches', ['enable-automation'])
    browser = webdriver.Chrome(options=options)
    browser.maximize_window()
    return browser


def clear_html_re(src_html):
    '''
    正则清除HTML标签
    :param src_html:原文本
    :return: 清除后的文本
    '''
    content = re.sub(r"</?(.+?)>", "", src_html) # 去除标签
    # content = re.sub(r"&nbsp;", "", content)
    dst_html = re.sub(r"\s+", "", content)  # 去除空白字符
    return dst_html


def scroll_to(browser, x=0, y=0):
        """浏览器内容水平滚动到指定位置x,垂直滚动到指定位置y
        @param browser: 浏览器实例对象
        @param x: 水平滚动到的位置x
        @param y: 垂直滚动到的位置y"""
        script = """
            (function() {
                window.scroll(%(x)s, %(y)s);
            })();""" % {'x': str(x), 'y': str(y), }
        try:
            browser.execute_script(script=script)
        except Exception as e:
            print("[scroll] Browser scroll to (%s, %s) failed" % (x, y))


def scroll_to_bottom(browser, step=100, delay=1000, currentY=0):
        """浏览器内容垂直滚动到最底部
        @param browser: 浏览器实例对象
        @param step: 滚动步长
        @param delay: 滚动延迟时间
        @param currentY: 当前垂直滚动到的位置"""
        
        script = """
            (function () {
                var y = %(currentY)s;
                var step = %(step)s;
                window.scroll(0, y);
                
                function f() {
                    if (y < document.body.scrollHeight) {
                        y += step;
                        window.scroll(0, y);
                        setTimeout(f, %(delay)s);
                    } else {
                        window.scroll(0, %(position)s);
                        document.title += "scroll-done";
                    }
                }
                
                setTimeout(f, %(delay)s);
            })();
        """ % {'step': step, 'delay': delay, 'currentY': currentY, 'position': "document.body.scrollHeight", }
        
        try:
            browser.execute_script(script=script)
        except Exception as e:
            print("[scroll] Browser scroll bottom failed" % step)


def get_scroll_range(browser):
        """获取浏览器可滚动范围
        @param browser: 浏览器实例对象
        @return: 浏览器滚动横向、纵向滚动范围"""
        # 获取浏览器滚动距离
        js = "var scrollWidth = document.body.scrollWidth; return(scrollWidth);"
        scroll_width = browser.execute_script(js)
        js = "var scrollHeight = document.body.scrollHeight; return(scrollHeight);"
        scroll_height = browser.execute_script(js)
        return (scroll_width, scroll_height,)


# import win32api
# win32api.ShellExecute(0, "open", r"D:\install\fiddler4\Fiddler\Fiddler.exe", "", "", 1)

# browser = init_chrome_browser()

# browser.get("https://sycm.taobao.com/")
# browser.implicitly_wait(50)

# # 关闭fidder


In [7]:
# 打开fiddler
# import win32api
# win32api.ShellExecute(0, "open", r"D:\install\fiddler4\Fiddler\Fiddler.exe", "", "", 1)

browser = init_chrome_browser()
browser.get("https://www.taobao.com/")
browser.implicitly_wait(50)


# 手动关闭fiddler

In [10]:
# 点击店铺
element2 = get_element(browser=browser, by=By.XPATH, value="//a[@class='ebase-Selector__header']")
js_click = """arguments[0].click();"""
browser.execute_script(js_click, element2)

# 选择店铺
elements = get_element(browser=browser, by=By.XPATH, value="//div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span", multi=True)
print(len(elements))

len_ele = len(elements)
print(len_ele)
ind = 0


db = connect()
cursor = db.cursor()

while True:
    index = 2 + ind
    print(index)
    store_name = elements[index].text.replace(u'...', '').strip()
    print(store_name)
    js_click = """arguments[0].click();"""
    browser.execute_script(js_click, elements[index])
    
    
    # 重新加载
    element2 = get_element(browser=browser, by=By.XPATH, value="//a[@class='ebase-Selector__header']")
    
    js_click = """arguments[0].click();"""
    browser.execute_script(js_click, element2)

    elements = get_element(browser=browser, by=By.XPATH, value="//div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span", multi=True)
        
    
    
    pay_amount_button = get_element(browser=browser, by=By.XPATH, value="//div[@class='index-cell payAmt oui-clearfix']//p[@class='index-value text-overflow']")
    count_date_button = get_element(browser=browser, by=By.XPATH, value="//span[@class='updateTime updateTime oui-pull-right']")
    
    pay_amt = pay_amount_button.text.strip().replace(",", "").replace(" ", "")
    count_date = count_date_button.text.strip().split(" ") and count_date_button.text.strip().split(" ")[0].replace('更新时间：', "") or time.strftime('%Y-%m-%d',time.localtime(time.time()))
    
    value = (store_name, count_date, pay_amt)
    
    print("##" * 40)
    print("[pay_amt] =", pay_amt)
    print("[count_date] =", count_date)
    print("##" * 40)
    
    sql = """INSERT INTO ds_ali_shop_realtime_payment(store_name, count_date, pay_amt)VALUES(%s,%s, %s)"""
    
    try:
        cursor.execute(sql, value)
    except Exception as e:
        print(e)

    db.commit()
    
    
    ind += 1
    
    if index == len_ele - 1:
        break


cursor.close() #关闭连接

db.close()




[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//a[@class='ebase-Selector__header']")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //a[@class='ebase-Selector__header']) successfull!
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span) successfull!
646
646
2
cartelo诺森专卖店
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//a[@class='ebase-Selector__header']")
[BrowserHe

[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//a[@class='ebase-Selector__header']")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //a[@class='ebase-Selector__header']) successfull!
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span) successfull!
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//div[@class='index-cell payAmt oui-clearfix']//p[@class='index-value te

[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //div[@class='index-cell payAmt oui-clearfix']//p[@class='index-value text-overflow']) successfull!
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//span[@class='updateTime updateTime oui-pull-right']")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //span[@class='updateTime updateTime oui-pull-right']) successfull!
################################################################################
[pay_amt] = 619
[count_date] = 2019-04-11
################################################################################
11
卡帝乐鳄鱼欢乐购专卖店
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//a[@class='ebase-Selector__header']")
[BrowserHelper

[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//a[@class='ebase-Selector__header']")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //a[@class='ebase-Selector__header']) successfull!
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span) successfull!
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//div[@class='index-cell payAmt oui-clearfix']//p[@class='index-value te

################################################################################
[pay_amt] = 2406
[count_date] = 2019-04-11
################################################################################
20
南极人纳梵专卖店
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//a[@class='ebase-Selector__header']")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //a[@class='ebase-Selector__header']) successfull!
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-t

...

################################################################################
[pay_amt] = 41364
[count_date] = 2019-04-11
################################################################################
216
南极人羽绒服品牌店
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//a[@class='ebase-Selector__header']")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //a[@class='ebase-Selector__header']) successfull!
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-ou

################################################################################
[pay_amt] = 6670
[count_date] = 2019-04-11
################################################################################
221
南极人丽美特专卖店
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//a[@class='ebase-Selector__header']")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //a[@class='ebase-Selector__header']) successfull!
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui

################################################################################
[pay_amt] = 0
[count_date] = 2019-04-11
################################################################################
226
静帆母婴专营店
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//a[@class='ebase-Selector__header']")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //a[@class='ebase-Selector__header']) successfull!
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-type

[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//a[@class='ebase-Selector__header']")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //a[@class='ebase-Selector__header']) successfull!
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span")
[BrowserHelper :: get_element] waite for load element ... [try_times] = 1 [element] = True
[BrowserHelper :: get_element] get element(xpath = //div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span) successfull!
[BrowserHelper :: get_element] 加载WebElement元素 ... [try_times] = 1 [wait_seconds] = 0.0 [locator] = ('xpath', "//div[@class='index-cell payAmt oui-clearfix']//p[@class='index-value te

AttributeError: 'NoneType' object has no attribute 'text'

'2019-04-11'